<a href="https://colab.research.google.com/github/julialucchesi1/Paralel_port/blob/main/Paralel_port.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
from multiprocessing import Process, Manager
from datetime import datetime

# Lista de ações do Dow Jones
dowjones = [
    'AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CSCO', 'CVX', 'DIS', 'DOW', 'GS',
    'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KO', 'MCD', 'MMM', 'MRK',
    'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'V', 'VZ', 'WBA', 'WMT', 'RTX'
]

# Parâmetros do período
start_date = '2024-08-01'
end_date = '2024-12-31'

# Baixando os dados de fechamento
data = yf.download(dowjones, start=start_date, end=end_date)['Close']

# Calcula retornos diários
returns = data.pct_change().dropna()

# Função de análise individual da carteira
def analise(i, results):
    np.random.seed(i)  # Para reprodutibilidade
    selected_stocks = random.sample(dowjones, 25)
    sub_returns = returns[selected_stocks]

    # Gera pesos aleatórios respeitando as restrições
    while True:
        weights = np.random.uniform(0, 0.2, 25)
        weights = weights / weights.sum()
        if all(weights <= 0.2) and weights.sum() <= 1:
            break

    # Retorno esperado da carteira
    mean_returns = sub_returns.mean()
    portfolio_return = np.dot(weights, mean_returns) * 252  # Anualizado

    # Matriz de covariância
    cov_matrix = sub_returns.cov()
    portfolio_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix * 252, weights)))  # Anualizada

    # Índice de Sharpe (sem risk-free rate)
    sharpe_ratio = portfolio_return / portfolio_std_dev if portfolio_std_dev != 0 else 0

    results.append({
        'index': i,
        'stocks': selected_stocks,
        'weights': weights.tolist(),
        'return': portfolio_return,
        'std_dev': portfolio_std_dev,
        'sharpe_ratio': sharpe_ratio
    })

# Multiprocessamento
if __name__ == '__main__':
    manager = Manager()
    results = manager.list()
    processes = [Process(target=analise, args=(i, results)) for i in range(1000)]

    for p in processes:
        p.start()
    for p in processes:
        p.join()

    # Converte resultados para DataFrame
    df_results = pd.DataFrame(list(results))
    df_results.to_csv('carteiras_analise.csv', index=False)
    print(df_results.head())



Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:130: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py:11211: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:2773: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/

   index                                             stocks  \
0      0  [IBM, TRV, AMGN, VZ, WMT, INTC, AAPL, MSFT, AX...   
1      1  [CAT, JPM, MRK, WBA, CVX, KO, AXP, V, HON, TRV...   
2      3  [KO, HD, CAT, JNJ, JPM, GS, DIS, RTX, HON, WMT...   
3      2  [JPM, AMGN, AXP, DIS, IBM, MMM, VZ, WMT, PG, C...   
4      7  [KO, IBM, TRV, MMM, MSFT, UNH, HD, RTX, GS, MC...   

                                             weights  return  std_dev  \
0  [0.03715943487232841, 0.04842452397930974, 0.0...     NaN      NaN   
1  [0.0379222714043285, 0.06550335625330218, 1.04...     NaN      NaN   
2  [0.051321492130280165, 0.06598282734137571, 0....     NaN      NaN   
3  [0.04425003616668324, 0.0026313075919322335, 0...     NaN      NaN   
4  [0.00593198093353816, 0.060628582336631556, 0....     NaN      NaN   

   sharpe_ratio  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  
